# AWS Data Lake: construindo pipelines na AWS

In [ ]:
!mkdir -p data

In [ ]:
import urllib.request

In [ ]:
def extract_data(url, filename):
  try:
    urllib.request.urlretrieve(url, filename)
  except Exception as e:
    print(e)

In [ ]:
extract_data("https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/c9509ab4-6f6d-4b97-979a-0cf2a10c922b/download/311_service_requests_2015.csv", "data/dados_2015.csv")
extract_data("https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/b7ea6b1b-3ca4-4c5b-9713-6dc1db52379a/download/311_service_requests_2016.csv", "data/dados_2016.csv")
extract_data("https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/30022137-709d-465e-baae-ca155b51927d/download/311_service_requests_2017.csv", "data/dados_2017.csv")
extract_data("https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/2be28d90-3a90-4af1-a3f6-f28c1e25880a/download/311_service_requests_2018.csv", "data/dados_2018.csv")
extract_data("https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/ea2e4696-4a2d-429c-9807-d02eb92e0222/download/311_service_requests_2019.csv", "data/dados_2019.csv")
extract_data("https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/6ff6a6fd-3141-4440-a880-6f60a37fe789/download/script_105774672_20210108153400_combine.csv", "data/dados_2020.csv")

In [ ]:
#Lita dos nomes dos arquivos baixados
arquivos = [
    "data/dados_2015.csv",
    "data/dados_2016.csv",
    "data/dados_2017.csv",
    "data/dados_2018.csv",
    "data/dados_2019.csv",
    "data/dados_2020.csv",
]

In [ ]:
#Dicionario para armazenar os dados de cada arquivo
dfs = {}

In [ ]:
import pandas as pd

#Loop para ler cada arquivo e adicionar ao dicionario
for arquivo in arquivos:
  ano = arquivo.split("_")[-1].split(".")[0] #Extrai o nome do arquivo
  dfs[ano] = pd.read_csv(arquivo)
  #Le os dados do arquivo transforma em dataframe e armazena no dicionario

In [ ]:
#Exemplo de como acessar os dados de um ano específico
dfs["2018"].head()

,case_enquiry_id,open_dt,target_dt,closed_dt,ontime,case_status,closure_reason,case_title,subject,reason,...,police_district,neighborhood,neighborhood_services_district,ward,precinct,location_street_name,location_zipcode,latitude,longitude,source
0,101002296861,2018-01-01 00:08:00,NaN,2018-03-01 15:18:12,ONTIME,Closed,Case Closed. Closed date : 2018-03-01 15:18:12...,Fire Hydrant,Boston Water & Sewer Commission,Fire Hydrant,...,E13,Jamaica Plain,11,Ward 19,1903,413 Centre St,2130.0,42.3214,-71.1109,Constituent Call
1,101002296862,2018-01-01 00:09:46,2018-01-02 08:30:00,2018-01-02 21:15:22,OVERDUE,Closed,Case Closed. Closed date : 2018-01-02 21:15:22...,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,...,C6,South Boston / South Boston Waterfront,5,Ward 6,0604,428 W Second St,2127.0,42.3373,-71.0445,Citizens Connect App
2,101002296864,2018-01-01 00:11:24,2018-01-02 08:30:00,2018-01-02 21:15:11,OVERDUE,Closed,Case Closed. Closed date : 2018-01-02 21:15:11...,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,...,C6,South Boston / South Boston Waterfront,5,Ward 6,0604,401-403 W Second St,2127.0,42.3371,-71.0442,Citizens Connect App
3,101002296865,2018-01-01 00:19:06,2018-01-02 08:30:00,2018-01-02 21:14:48,OVERDUE,Closed,Case Closed. Closed date : 2018-01-02 21:14:48...,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,...,B3,Dorchester,9,17,1701,INTERSECTION School St & Washington St,NaN,42.3594,-71.0587,Citizens Connect App
4,101002296866,2018-01-01 00:21:25,2018-01-02 08:30:00,2018-01-02 21:14:32,OVERDUE,Closed,Case Closed. Closed date : 2018-01-02 21:14:32...,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,...,B3,Dorchester,9,Ward 17,1701,6-8 School St,2124.0,42.2965,-71.0729,Citizens Connect App


## Inserindo arquivo em bucket S3

In [ ]:
pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.6/82.6 kB 5.4 MB/s eta 0:00:00


In [ ]:
import boto3

In [ ]:
#Importante: essas credenciais nao podem aparecer no seu codigo coloque em variaveis de ambiente.
#Exemplo: aws_access_key_id = os.environ["a_sua_chave_"]

aws_access_key_id ="----"
aws_secret_access_key = "----"
region_name = "us-east-1"

In [ ]:
#Utilizando boto3 e estabelecendo uma sessão padrão passando os parametros.

boto3.setup_default_session(
    aws_access_key_id = aws_access_key_id,
    aws_secret_access_key = aws_secret_access_key,
    region_name = region_name
)

#Criacao de um cliente S3
s3 = boto3.client("s3")

In [ ]:
# Cria arquivo teste para ver a conexão com S3 foi feita com sucesso.
content = """
Olá, S3
"""

# Cria de fato o arquivo. E escreve as informações.
with open("hello-s3.txt", "w+") as f:
  f.write(content)

In [ ]:
bkt = "alura-datalakeaws99777"

In [ ]:
# Salva o arquivo no bucket S3 na AWS. NOME, BUCKET E PASTA
s3.upload_file("hello-s3.txt", bkt, "bronze/hello-s3")

## Salvando arquivo em Parquet

In [ ]:
from io import BytesIO

In [ ]:
for ano, df in dfs.items():
  parquet_buffer = BytesIO()
  df.to_parquet(parquet_buffer)

  #Metodo do S3 para gravar objetos dentro do S3.
  s3.put_object(
      Bucket=bkt,
      Key=f"bronze/dados_{ano}.parquet",
      Body=parquet_buffer.getvalue()
  )

In [ ]:
# Obtendo a lista de objetos do bucket.
response = s3.list_objects(Bucket=bkt)

In [ ]:
response

{'ResponseMetadata': {'RequestId': 'G8G5Z0A6DYYJAR1S',
  'HostId': 'rLUF6u9VTM0i6CnzoGMK4zgt0yo3kR0NOThVP2Ry/yZ6I38MQ+ncx1kk5yQ244TgNACSVmaQyyDhwZK+NctNSoT3rUW3sCXa',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'rLUF6u9VTM0i6CnzoGMK4zgt0yo3kR0NOThVP2Ry/yZ6I38MQ+ncx1kk5yQ244TgNACSVmaQyyDhwZK+NctNSoT3rUW3sCXa',
   'x-amz-request-id': 'G8G5Z0A6DYYJAR1S',
   'date': 'Sat, 26 Oct 2024 18:49:39 GMT',
   'x-amz-bucket-region': 'us-east-1',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'IsTruncated': False,
 'Marker': '',
 'Contents': [{'Key': 'bronze/',
   'LastModified': datetime.datetime(2024, 10, 25, 21, 33, 34, tzinfo=tzlocal()),
   'ETag': '"d41d8cd98f00b204e9800998ecf8427e"',
   'Size': 0,
   'StorageClass': 'STANDARD',
   'Owner': {'DisplayName': 'menezes_kennedy',
    'ID': '364555334ca7c1059c50f0a58b89ebcab6c5366aeac53c9c60e0f9a6c4d72da2'}},
  {'Key': 'bronze/dados_2015.parquet',
   'LastModifie

In [ ]:
#Obtendo a lista de chaves dos objetos do bucket
keys = [obj["Key"] for obj in response ["Contents"]]
print(keys)

['bronze/', 'bronze/dados_2015.parquet', 'bronze/dados_2016.parquet', 'bronze/dados_2017.parquet', 'bronze/dados_2018.parquet', 'bronze/dados_2019.parquet', 'bronze/dados_2020.parquet', 'silver/']
